In [2]:
!pip install nba_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.2/257.2 KB 8.3 MB/s eta 0:00:00


In [9]:
import requests
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder
from sklearn.ensemble import RandomForestClassifier
import datetime

# Функция для получения ближайших матчей
def get_upcoming_games(date):
    url = f"https://api-nba-v1.p.rapidapi.com/games/date/{date}"
    headers = {
        "X-RapidAPI-Key": "ad416b7619msheb40c8f58c5ad22p1c9c70jsn9004945f260a",
        "X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers)
    print(response)
    games = response.json()["api"]["games"]
    upcoming_games = []

    for game in games:
        if game["statusShortGame"] == "NS":  # NS означает "Not Started"
            upcoming_games.append((game["gameId"], game["hTeam"]["teamId"], game["vTeam"]["teamId"]))

    return upcoming_games

# Функция для получения статистики игроков команды
def get_team_stats(team_id):
    team_stats = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id).get_data_frames()[0]
    return team_stats

# Функция для обучения модели на основе имеющихся данных
def train_model(team_stats_home, team_stats_away):
    dataset = pd.concat([team_stats_home, team_stats_away])
    X = dataset.drop(["GAME_ID", "TEAM_ID", "WL"], axis=1)
    y = dataset["WL"]

    clf = RandomForestClassifier()
    clf.fit(X, y)
    return clf

# Получение текущей даты и даты через 7 дней
today = datetime.date.today()
one_week_later = today + datetime.timedelta(days=7)

# Получение ближайших матчей для каждой даты в промежутке
upcoming_games = []
for date in pd.date_range(today, one_week_later):
    upcoming_games.extend(get_upcoming_games(date.strftime("%Y-%m-%d")))

# Анализ и предсказание результатов матчей
for game_id, home_team_id, away_team_id in upcoming_games:
    team_stats_home = get_team_stats(home_team_id)
    team_stats_away = get_team_stats(away_team_id)

    model = train_model(team_stats_home, team_stats_away)

    # Получение последних статистик обеих команд
    last_game_home = team_stats_home.iloc[-1].drop(["GAME_ID", "TEAM_ID", "WL"])
    last_game_away = team_stats_away.iloc[-1].drop(["GAME_ID", "TEAM_ID", "WL"])

    # Предсказание результатов матча
    home_win_probability = model.predict_proba([last_game_home])[0][1]
    away_win_probability = model.predict_proba([last_game_away])[0][0]

    print(f"Вероятность победы домашней команды: {home_win_probability * 100:.2f}%")
    print(f"Вероятность победы гостевой команды: {away_win_probability * 100:.2f}%")

    if home_win_probability > away_win_probability:
        print(f"Домашняя команда с ID {home_team_id} имеет больше шансов на победу.")
    elif away_win_probability > home_win_probability:
        print(f"Гостевая команда с ID {away_team_id} имеет больше шансов на победу.")
    else:
        print("Шансы на победу обеих команд равны.")

    print("\n")


<Response [429]>


KeyError: ignored